In [1]:
#Spark context
sc = SparkContext.getOrCreate()

In [2]:
#Read data
rdd=sc.textFile('input.txt')
print('line numbers=',rdd.count())

line numbers= 5


In [3]:
#clean data, e.g., filter empty spaces and convert all to lower case
def lowerCase(line):
    values=[x.lower() for x in line]
    return(values)
    
rddClean=rdd.map(lambda line: line.split(" ")).map(lowerCase).filter(lambda w: w if (w!='') else None)


In [4]:
#Generate key pairs (co-occurrence) & frequency for each sentence
import itertools
from operator import add

def pairLines(w):
    values = [x for x in itertools.combinations(w, 2)]
    return values

rddPairsFreq=rddClean.map(pairLines).flatMap(lambda w: w).map(lambda w: (frozenset([max(w[0],w[1]),min(w[0],w[1])]),1)).reduceByKey(add)


In [5]:
# Make dictionary from documnet (all sentences) and compute frequency
rddDictFreq=rddClean.flatMap(lambda w:w).map(lambda w:(w,1)).reduceByKey(add)


In [6]:
# Compute Jaccard
rddJaccard_=rddPairsFreq.map(lambda w: [list(w[0])[0],list(w[0])[1],w[1]]).map(lambda w: (w[0],w[1:])).join(rddDictFreq)
rddJaccard=rddJaccard_.map(lambda w: (w[1][0][0],[w[0],w[1][0][1],w[1][1]])).join(rddDictFreq).map(lambda x: [x[0],x[1][0][0],x[1][0][1]/max(x[1][0][2],x[1][1])])
rddJaccard.collect()

[['julies', 'favorite', 0.3333333333333333],
 ['julies', '', 0.3333333333333333],
 ['julies', 'a', 0.3333333333333333],
 ['julies', 'blue', 0.3333333333333333],
 ['julies', 'than', 0.3333333333333333],
 ['julies', 'better', 0.3333333333333333],
 ['friend', 'the', 1.0],
 ['friend', 'red', 0.3333333333333333],
 ['friend', 'a', 0.5],
 ['friend', 'great', 1.0],
 ['friend', 'is', 0.3333333333333333],
 ['friend', 'dog', 0.2],
 ['big', 'the', 0.5],
 ['big', 'red', 0.6666666666666666],
 ['big', 'a', 0.5],
 ['big', 'great', 0.5],
 ['big', 'favourite', 0.5],
 ['big', 'animal', 0.5],
 ['big', 'is', 0.3333333333333333],
 ['big', 'clifford', 0.5],
 ['big', 'dog', 0.4],
 ['big', 'friend', 0.5],
 ['the', 'a', 0.5],
 ['the', 'great', 1.0],
 ['the', 'dog', 0.2],
 ['is', 'favorite', 0.3333333333333333],
 ['is', 'the', 0.3333333333333333],
 ['is', 'red', 0.3333333333333333],
 ['is', 'a', 0.6666666666666666],
 ['is', 'great', 0.3333333333333333],
 ['is', 'animal', 0.3333333333333333],
 ['is', 'blue', 0.33